In [ ]:
#!pip install cx-Oracle

Descrição: 
Essa demanda era realizada manualmente pelos analistas e demandava tempo devido a quantidade de informações. Era recuperado informações de 5 scrips SQL em base de dados Oracle para posterior armazenamento em planilha excel. 

Como as consultas eram no âmbito do Exército Brasileiro, as informações tinham que ser divididas de acordo com a responsabilidade de cada Comando Militar de Área e Grande Unidade para envio periódico. 

Os analistas demoravam alguns dias para organizar todos os arquivos. Após a implementação desse código o trabalho passou a ser entregue em minutos.


Abaixo um breve resumo da quantidade de arquivos que eram gerados:

Comandos Militares: 8
Grandes Unidades: 10

Quantidade de arquivos gerados: 18
Quantidade de abas por Arquivo: 6 (5 abas das informações e 1 para a legenda dos dados)
Quantidade de abas ao total: 108







In [ ]:
import cx_Oracle
from getpass import getpass
import pandas as pd

In [ ]:
#inicia a conexão com o banco de dados Oracle
try:
    conexao = cx_Oracle.connect(
        user = '<user>',
        password = '<pass>',
        dsn = '<dsn>')
except Exception as err:
    print('Erro: ',err)
else:
    print('Conectado: ', conexao.version)

In [ ]:
# Diretório aonde os script SQL de consultas estão armazenados
diretorio_sql = "<diretorio>"

In [ ]:
#Nome dos dos arquivos .sql utilizados
lista_sql = ['script1.sql',
             'script2.sql',
             'script3.sql',
             'script4.sql',
             'script5.sql']

In [ ]:
#Arquivo que consta explicações das informações extraídas. Será armazenado na última aba de cada planilha gerada.
legenda_roteiro = 'legenda_abas.txt'

In [ ]:
cursor = conexao.cursor()

for arquivo_sql in lista_sql:
    #lê o arquivo .sql
    with open(f'{diretorio_sql}{arquivo_sql}', "r") as file:
        #armazena o script na variável
        sql = file.read()
        #executa o sql no banco de dados
        cursor.execute(sql)
        #armazena os resultados na variável
        row = cursor.fetchall()

        if arquivo_sql == "script1.sql":
            assunto_1 = pd.DataFrame(row, columns = ['Col_1','Col_2','Col_3','Col_4', 'Col_5', 'Qtd'])
        elif arquivo_sql == "script2.sql":
            assunto_2 = pd.DataFrame(row, columns = ['Col_1','Col_2','Col_3','Col_4', 'Col_5', 'Qtd'])
        elif arquivo_sql == "script3.sql":
            assunto_3 = pd.DataFrame(row, columns = ['Col_1','Col_2','Col_3','Col_4', 'Col_5', 'Qtd'])
        elif arquivo_sql == "script4.sql":
            assunto_4 = pd.DataFrame(row, columns = ['Col_1','Col_2','Col_3','Col_4', 'Col_5', 'Qtd'])
        elif arquivo_sql == "script5.sql":
            assunto_5 = pd.DataFrame(row, columns = ['Col_1','Col_2','Col_3','Col_4', 'Col_5', 'Qtd_1', 'Qtd_2','Qtd_3','Qtd_4','Qtd_5', 'Qtd_5', 'Qtd_7', 'Qtd_8'])

cursor.close()
display("Finalizado")

In [ ]:
#encerra a conexão com o banco de dados
conexao.close()

In [ ]:
#criação de log para verificar se o total de registros de cada script está correto
log = []
log.append(f"Total assunto 1: {len(assunto_1)}\t")
log.append(f"Total assunto 2: {len(assunto_2)}")
log.append(f"Total assunto 3: {len(assunto_3)}")
log.append(f"Total assunto 4: {len(assunto_4)}")
log.append(f"Total assunto 5: {len(assunto_5)}\n\n\n")

In [ ]:
#Lista com a sigla dos Comandos Militares existentes no Brasil
lista_cmdo_mil_a = ['CMSE', 'CMNE', 'CMS', 'CMA', 'CMP', 'CMO', 'CML', 'CMN']

In [ ]:
relatorios = [assunto_1, assunto_2, assunto_3, assunto_4, assunto_5]

In [ ]:
#Lista com o descrição de cada aba das planilhas
abas = ["aba_1","aba_2","aba_3","aba_4","aba_5"]

In [ ]:
# Para cada Comando Militar de Área será gerado uma planilha contendo 6 abas. 5 referente aos assuntos e 1 para a legenda
for cmdo_mil_a in lista_cmdo_mil_a:
    #variável utilizada para leitura de cada posição da lista de relatórios
    i = 0
    # Usando o ExcelWriter, cria um documento .xlsx, usando engine='xlsxwriter'
    writer = pd.ExcelWriter(f"Relatório - {cmdo_mil_a}.xlsx", engine='xlsxwriter')
    
    #Recuperação do nome de cada aba que constará na planilha
    for aba in abas:  
        #Descrição da aba é armazenada na variável
        rel = relatorios[i]
        #filtra os dados a serem armazenados na planilha
        rel_filtrado = rel[rel["Cmdo Mil A"] == cmdo_mil_a]
        #armazena os dados na planilha e define o nome da aba
        rel_filtrado.to_excel(writer, sheet_name = aba, index = False)
        i += 1
    #abertura do arquivo de texto para inserção da legenda na última aba
    with open(f'{diretorio_sql}{legenda_roteiro}', "r") as file:
        lines = file.readlines()
    df = pd.DataFrame(lines, columns=['Roteiro Legenda'])
    df.to_excel(writer, sheet_name = "Legenda", index = False)

    #salva os dados no arquivo
    writer.save()
display('Finalizado')

In [ ]:
lista_gu = ['GU_1','GU_2','GU_3', 'GU_4', 'GU_5', 'GU_6', 'GU_7', 'GU_8', 'GU_9','GU_10']

In [ ]:
# Para cada Grande Unidade será gerado uma planilha contendo 6 abas. 5 referente aos assuntos e 1 para a legenda
for gu in lista_gu:
    #variável utilizada para leitura de cada posição da lista de relatórios
    i = 0
    # Usando o ExcelWriter, cria um documento .xlsx, usando engine='xlsxwriter'
    writer = pd.ExcelWriter(f"Relatório - {gu}.xlsx", engine='xlsxwriter')

    #Recuperação do nome de cada aba que constará na planilha
    for aba in abas:
        #Descrição da aba é armazenada na variável
        rel = relatorios[i]
        #filtra os dados a serem armazenados na planilha
        rel_filtrado = rel[rel["GU"] == gu]
        #armazena os dados na planilha e define o nome da aba
        rel_filtrado.to_excel(writer, sheet_name = aba, index = False)
        i += 1
        
    #abertura do arquivo de texto para inserção da legenda na última aba
    with open(f'{diretorio_sql}{legenda_roteiro}', "r") as file:
        lines = file.readlines()
    df = pd.DataFrame(lines, columns=['Roteiro Legenda'])
    df.to_excel(writer, sheet_name = "Legenda", index = False)

    #salva os dados no arquivo
    writer.save()
display('Finalizado')

In [ ]:
#Arquivo de log para armazenar a quantidade de registros recuperados em cada script SQL 
with open('log_arquivos.txt', 'w', encoding='utf-8') as f:
    for i in log:
        f.write(f'{i}\n')
f.close()
display("Finalizado")